In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
# Define the date range
date_range = pd.date_range(start='2019-01-01', end='2023-12-01', freq='MS')

In [ ]:
# Generate more varied balances using random walk
np.random.seed(42)
balances = 100000 + np.random.randn(len(date_range)).cumsum() * 5000 + np.linspace(0, 200000, len(date_range))

In [ ]:
# Create the DataFrame
df = pd.DataFrame({'Date': date_range, 'Balance': balances})

In [ ]:

# Function to create dataset with look-back for LSTM 
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)] 
        X.append(a)
        Y.append(dataset[i + look_back])
    return np.array(X), np.array(Y)

In [ ]:
# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
df['Balance'] = scaler.fit_transform(df['Balance'].values.reshape(-1, 1))

In [ ]:
# Prepare the dataset for LSTM
look_back = 12
X, Y = create_dataset(df['Balance'].values, look_back)

In [ ]:
# Split the dataset into training and testing sets (80-20 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [ ]:
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])) 
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(1, look_back))) 
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=100, batch_size=1, validation_data=(X_test, Y_test), verbose=1)

In [ ]:
# Predict for each month in 2024
months_in_2024 = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')

In [ ]:
# Prepare data for predictions
X_future = df[-look_back:].drop('Date', axis=1).values.reshape(1, 1, look_back) # Use last look_back months from original data

In [ ]:
# Predict using the model
Y_future = []
predictions_2024_lstm = pd.DataFrame(columns=['Date', 'Predicted_Balance']) # Initialize DataFrame for predictions
for i in range(len(months_in_2024)):
    predicted_value = model.predict(X_future)
    Y_future.append(predicted_value[0, 0])
    X_future = np.append(X_future[:, :, 1:], predicted_value).reshape(1, 1, look_back)
    # Append predicted balance and corresponding date to DataFrame
    new_prediction = pd.DataFrame({'Date': [months_in_2024[i]], 'Predicted_Balance': [predicted_value[0, 0]]}) 
    predictions_2024_lstm = pd.concat([predictions_2024_lstm, new_prediction], ignore_index=True)


In [ ]:
# Inverse transform the predicted values to original scale
Y_future = scaler.inverse_transform(np.array(Y_future).reshape(-1, 1))
# Print or use Y_future as needed for predictions for each month in 2024 
for month, balance in zip(months_in_2024, Y_future):
    print(f"LSTM Predicted balance for {month.strftime('%Y-%m')}: ${balance[0]:,.2f}") 

In [ ]:
# Evaluate LSTM on test data
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1)) 
y_test_inv = scaler.inverse_transform(Y_test.reshape(-1, 1))

In [ ]:
# Calculate MSE, MAE, and RMSE for LSTM
mse_lstm = mean_squared_error(y_test_inv, y_pred_inv) 
mae_lstm = mean_absolute_error(y_test_inv, y_pred_inv) 
rmse_lstm = np.sqrt(mse_lstm)
print(f"LSTM Mean Squared Error (MSE) on test data: {mse_lstm:.2f}") 
print(f"LSTM Mean Absolute Error (MAE) on test data: {mae_lstm:.2f}") 
print(f"LSTM Root Mean Squared Error (RMSE) on test data: {rmse_lstm:.2f}")

In [ ]:
# Exponential Smoothing
model_es = ExponentialSmoothing(df['Balance'], seasonal='add', seasonal_periods=12).fit()

In [ ]:
# Predict for each month in 2024 using Exponential Smoothing 
predictions_2024_es = model_es.forecast(steps=len(months_in_2024))
# Inverse transform the predictions (since the original data was normalized) 
predictions_2024_es_inv = scaler.inverse_transform(predictions_2024_es.values.reshape(-1, 1))
# Print or use predictions_2024_es_inv as needed for predictions for each month in 2024 
for month, balance in zip(months_in_2024, predictions_2024_es_inv):
    print(f"Exponential Smoothing Predicted balance for {month.strftime('%Y-%m')}: ${balance[0]:,.2f}")

In [ ]:
# Evaluate Exponential Smoothing on test data
y_test_es_inv = scaler.inverse_transform(model_es.fittedvalues[-len(Y_test):].values.reshape(-1, 1))

# Calculate MSE, MAE, and RMSE for Exponential Smoothing 
mse_es = mean_squared_error(y_test_inv, y_test_es_inv) 
mae_es = mean_absolute_error(y_test_inv, y_test_es_inv) 
rmse_es = np.sqrt(mse_es)
print(f"Exponential Smoothing Mean Squared Error (MSE) on test data: {mse_es:.2f}") 
print(f"Exponential Smoothing Mean Absolute Error (MAE) on test data: {mae_es:.2f}") 
print(f"Exponential Smoothing Root Mean Squared Error (RMSE) on test data: {rmse_es:.2f}")


In [ ]:
# Visualize actual vs predicted for LSTM
plt.figure(figsize=(14, 7))
plt.plot(df['Date'][-len(y_test_inv):], y_test_inv, label='Actual') 
plt.plot(df['Date'][-len(y_test_inv):], y_pred_inv, label='LSTM Predicted') 
plt.title('Actual vs LSTM Predicted Balance')
plt.xlabel('Date') 
plt.ylabel('Balance')
plt.legend() 
plt.grid(True) 
plt.show()

In [ ]:

# Visualize predicted values for 2024 using LSTM
plt.figure(figsize=(14, 7))
plt.plot(df['Date'], df['Balance'], label='Historical Balance')

plt.plot(predictions_2024_lstm['Date'], predictions_2024_lstm['Predicted_Balance'], marker='o', linestyle='-', color='r', label='LSTM Predicted Balance')
plt.title('LSTM Predicted Balance for 2024')
plt.xlabel('Date') 
plt.ylabel('Balance') 
plt.legend() 
plt.grid(True) 
plt.show()


In [ ]:

# Visualize actual vs predicted for Exponential Smoothing
plt.figure(figsize=(14, 7))
plt.plot(df['Date'][-len(y_test_inv):], y_test_inv, label='Actual') 
plt.plot(df['Date'][-len(y_test_inv):], y_test_es_inv, label='Exponential Smoothing Predicted') 
plt.title('Actual vs Exponential Smoothing Predicted Balance')
plt.xlabel('Date') 
plt.ylabel('Balance') 
plt.legend() 
plt.grid(True) 
plt.show()


In [ ]:

# Visualize predicted values for 2024 using Exponential Smoothing
plt.figure(figsize=(14, 7))
plt.plot(df['Date'], df['Balance'], label='Historical Balance')
plt.plot(months_in_2024, predictions_2024_es_inv, marker='o', linestyle='-', color='g', label='Exponential Smoothing Predicted Balance for 2024')
plt.title('Exponential Smoothing Predicted Balance for 2024')
plt.xlabel('Date') 
plt.ylabel('Balance') 
plt.legend() 
plt.grid(True) 
plt.show()

In [ ]:

# Visualize actual vs predicted for Exponential Smoothing
plt.figure(figsize=(14, 7))
plt.plot(df['Date'][-len(y_test_inv):], y_test_inv, label='Actual') 
plt.plot(df['Date'][-len(y_test_inv):], y_test_es_inv, label='Exponential Smoothing Predicted') 
plt.title('Actual vs Exponential Smoothing Predicted Balance')
plt.xlabel('Date') 
plt.ylabel('Balance') 
plt.legend() 
plt.grid(True) 
plt.show()


In [ ]:

# Visualize predicted values for 2024 using Exponential Smoothing
plt.figure(figsize=(14, 7))
plt.plot(df['Date'], df['Balance'], label='Historical Balance')
plt.plot(months_in_2024, predictions_2024_es_inv, marker='o', linestyle='-', color='g', label='Exponential Smoothing Predicted Balance for 2024')
plt.title('Exponential Smoothing Predicted Balance for 2024')
plt.xlabel('Date') 
plt.ylabel('Balance') 
plt.legend() 
plt.grid(True) 
plt.show()